In [1]:
from cloud_spider import NWSpider

In [2]:
url = 'https://www.networkworld.com/cloud-computing/'
spider = NWSpider(url)
spider.extract()

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from datetime import datetime as dt
from sqlalchemy import create_engine
import os

In [2]:
class NWSpider():
    def __init__(self, url):
        conn_url = os.environ.get('POSTGRES_CONNECTION_STRING')
        self.engine = create_engine(f'{conn_url}cloudnewsdb')
        self.data = []
        self.url = url

        source = requests.get(url).text
        self.soup = BeautifulSoup(source, 'lxml')

    def extract(self):
        for a in self.soup.find_all('a', attrs={'class': 'card'}):
            date = a.find_next('div', attrs={'class': 'card__info--light'}).find_next('span').text
            if 'dec' in date.lower():
                self.data.append({
                    'title': a.find_next('h4').text,
                    'url': a['href'],
                    'date': date,
                    'source': 'Network World',
                    'query_date': dt.now()
                })
        df = pd.DataFrame(self.data)
        df.to_sql('raw', self.engine, if_exists='replace')

In [3]:
url = 'https://www.ciodive.com/topic/cloud/'
source = requests.get(url).text
soup = BeautifulSoup(source, 'lxml')

In [16]:
for li in soup.select('li.row.feed__item'):
    a = li.find_next('h3', attrs={'class': 'feed__title'}).find_next('a')
    url = a['href']
    title = a.text.strip()
    try:
        date = li.find_all('span', attrs={'class':'secondary-label'})[1].text
    except:
        print(li)
    if 'jan' in date.lower() or 'dec' in date.lower():
        print({
        'title':title,
        'url': url,
        'date': date,
        'source': 'Ciodive',
        'query_date': dt.now()
    })

{'title': 'SAP adds board-level cloud migration unit, reshuffles executive roles', 'url': '/news/sap-board-add-cloud-unit-saas-enterprise-software-shift/704273/', 'date': 'Jan. 11, 2024', 'source': 'Ciodive', 'query_date': datetime.datetime(2024, 1, 11, 21, 42, 37, 19071)}
{'title': 'Nvidia unveils laptop AI chip, developer toolkit', 'url': '/news/nvidia-laptop-pc-gpu-generative-ai-processor/704227/', 'date': 'Jan. 10, 2024', 'source': 'Ciodive', 'query_date': datetime.datetime(2024, 1, 11, 21, 42, 37, 20134)}
<li class="row feed__item">
<div class="medium-6 columns">
<figure class="feed__image-container feed__image-container-trendline">
<a class="feed__image" href="/trendline/hybrid-cloud/391/">
<img alt="A large cloud icon with smaller tech icons connected to the cloud icon" class="" height="675" sizes="(min-width: 40em ) 25vw, 100vw" src="/imgproxy/5XveLrgr9Ljur_JLOTxcWt3fpHUU8lQOb6oKKtLsilo/g:ce/rs:fill:1200:675:1/bG9jYWw6Ly8vZGl2ZWltYWdlL2Nsb3VkLWxvZ28tdjIuanBn.jpg" srcset="/imgpr